In [1]:
from openai import OpenAI,AsyncOpenAI
client = OpenAI(base_url="https://api.302.ai/v1/chat/completions", api_key="sk-gPl43py4bj6TN3mksIN18FPbRZXdHoV6i0XOAAshKFpanIb1")#ollama服务默认端口


In [2]:
@staticmethod
def get_weahter(location,food):
    summary=f"""位置：{location} 食物：{food}
    """
    return summary
def get_weahter(location,food):
   if location=="Shanghai":
       return "sunny"+"fish"
   elif location=="Beijing":
       return "rainy"+"fish"
   else:
       return f"I don't know the weahter of location {location}{food}"

#定义第二个工具的函数实现,该工具能够实现输入一个地名，返回一句话我已经订好票了并且在周四的时候去这个地方玩
def get_visiting_place(visiting_place):
    return f'I have order the ticket and we will go {visiting_place} on Thursday.'

#定义大模型调用工具的schema(工具的定义)
tools=[{
        "type": "function",
            "function": {
                "name": "get_weather",
                "description": "获取当前位置的天气",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "中国的省份名称的拼音表示,第一个字大写如:Sichuan",
                            "default":"未提及"
                        },
                        # "food": {
                        #     "type": "string",
                        #     "description": "当前所在的城市的食物，北京：老北京豆浆",
                        #     "default":"未提及"
                        # },
                    }
                },
                "required": ["location","food"] #必填参数
            }
    },
    {
        "type": "function",
            "function": {
            "name": "get_visiting_place",
            "description": "获取去访问的景点",
            "parameters": {
                "type": "object",
                "properties": {
                    "visiting_place": {
                        "type": "string",
                        "description": "景点名称",
                    },
                }
            },
            "required": ["visiting_place"]  # 必填参数
        },
    }]

In [3]:
# check_functions =  {"get_weather":get_weahter,
#                     "get_visiting_place":get_visiting_place}
response1 = client.chat.completions.create(
    model='gpt-4.1',
    messages=[
          {"role": "system", "content": "You are a helpful assistant"},
        # {"role": "user", "content": "我想去北京玩,天气是什么样的"},
           {"role": "user", "content": "我想去北京玩,请问北京的天气如何？我想去故宫玩"},
      ],
    stream=False , # 先将stream(流式输出)设置为False,
    tools=tools,
     )

# response1.choices[0].delta.tool_calls

In [4]:
def deepseek_(messages):
    response1 = client.chat.completions.create(
        model='gpt-4.1',
        messages=messages,
        stream=False , # 先将stream(流式输出)设置为False,
        tools=tools,
         )

In [5]:

check_functions =  {"get_weather":get_weahter,
                    "get_visiting_place":get_visiting_place}
if response1.choices[0].finish_reason=="tool_calls":
    for i in response1.choices[0].message.tool_calls:
        tool_name = i.function.name
        tool_args = i.function.arguments
        tool_args = eval(tool_args)
        tool_result = check_functions[tool_name](**tool_args)
        print("调用的工具:",tool_name,"槽位参数:",tool_args,"工具调用结果:",tool_result)
    for i in  response1.choices[0].delta.content:  # 检查是否有内容决定是function call还是不同回复
        a=response1.choices[0].delta.content
        print(a)
else:     #没有触发工具调用
    return_message=response1.choices[0].message.content
    print("未触发工具调用，直接回复:",return_message)
        

TypeError: get_weahter() missing 1 required positional argument: 'food'

In [16]:
from openai import OpenAI,AsyncOpenAI #导入异步客户端库
check_functions =  {"get_weather":get_weahter,
                    "get_visiting_place":get_visiting_place}
deepseek_async_client = AsyncOpenAI(base_url="https://api.302.ai/v1/chat/completions", api_key="sk-gPl43py4bj6TN3mksIN18FPbRZXdHoV6i0XOAAshKFpanIb1")
async def deepseek_steam_llm(tools,messages):
    
    stream =await deepseek_async_client.chat.completions.create(
        model="gpt-4.1",
        messages=messages,
        stream=True, #流式输出
        tools=tools
    )
    async for chunk in stream:
        yield chunk #注意使用yield输出,不能用return

In [24]:
import json
@staticmethod
def get_weahter(location,food):
    summary=f"""位置：{location} 食物：{food}
    """
    return summary
# def get_weahter(location,food):
#    if location=="Shanghai":
#        return "sunny"+"fish"
#    elif location=="Beijing":
#        return "rainy"+"fish"
#    else:
#        return f"I don't know the weahter of location {location}{food}"

#定义第二个工具的函数实现,该工具能够实现输入一个地名，返回一句话我已经订好票了并且在周四的时候去这个地方玩
def get_visiting_place(visiting_place):
    return f'I have order the ticket and we will go {visiting_place} on Thursday.'

#定义大模型调用工具的schema(工具的定义)
tools=[{
        "type": "function",
            "function": {
                "name": "get_weather",
                "description": "获取当前位置的天气和当地特色美食",
                "properties": {
                        "location": {
                            "type": "string",
                            "description": "中国的省份名称的拼音表示,第一个字大写如:Sichuan",
                        },
                        "food": {
                            "type": "string",
                            "description": "获取location的特色美食比如北京是老北京豆浆",
                            
                        },
                    },
                "required": ["location","food"] #必填参数
            }
    },
    {
        "type": "function",
            "function": {
            "name": "get_visiting_place",
            "description": "获取去访问的景点",
            "parameters": {
                "type": "object",
                "properties": {
                    "visiting_place": {
                        "type": "string",
                        "description": "景点名称",
                    },
                }
            },
            "required": ["visiting_place"]  # 必填参数
        },
    }]
tool_calls_dict={} #记录单个工具调用的结果
complete_tool_calls=[] #记录所有工具调用的结果

In [26]:
messages = [
        {"role": "system", "content": "You are a helpful assistant"},
        # {"role": "user", "content": "你好"},
        {"role": "user", "content": "我想去北京玩,请问北京的天气如何？有什么好吃的推荐"},
    ]
async for chunk in deepseek_steam_llm(tools, messages):
    # print("origin_chunk:",chunk)
    if chunk.choices:  # 元数据不要
        if chunk.choices[0].delta.tool_calls:
            for tool_call in chunk.choices[0].delta.tool_calls:
                
                tool_call_id = tool_call.id
                function_name = tool_call.function.name
                arguments_fragment = tool_call.function.arguments
                # print("arguments_fragment:", arguments_fragment)
                # 初始化工具调用信息 tool_call_id为None的话也不开一个新的key
                if tool_call_id not in tool_calls_dict and tool_call_id:
                    tool_calls_dict[tool_call_id] = {
                        "name": function_name,
                        "arguments": ""
                    }
                # print("触发functioncalling回复:", chunk.choices[0].delta.tool_calls)
                if tool_call_id and arguments_fragment:
                    tool_calls_dict[tool_call_id]["arguments"] += arguments_fragment
                #此时arguments_fragment为None
                elif not arguments_fragment:
                    continue
                else:
                    # 获取最后一个有key的值,
                    last_key_id = list(tool_calls_dict.keys())[-1]
                    tool_calls_dict[last_key_id]["arguments"] += arguments_fragment
        
        print(tool_calls_dict)
        if chunk.choices[0].delta.content:  # 检查是否有内容决定是function call还是不同回复
           # accumulate_message += chunk.choices[0].delta.content
            print("触发普通问答回复:", chunk.choices[0].delta.content)
            # print('///',chunk.choices[0].delta.content)

        if chunk.choices[0].finish_reason == "tool_calls":
            print("工具调用结束")
            for tool_call_id, tool_call_info in tool_calls_dict.items():
                print(tool_call_id, tool_call_info)

{'call_AfHEIXajrvBvU0H3x865Pefj': {'name': 'get_weather', 'arguments': '{}'}, 'call_sWp4ie35amsFKvReXAvQ1Mzg': {'name': 'get_weather', 'arguments': ''}}
{'call_AfHEIXajrvBvU0H3x865Pefj': {'name': 'get_weather', 'arguments': '{}'}, 'call_sWp4ie35amsFKvReXAvQ1Mzg': {'name': 'get_weather', 'arguments': '{}'}}
{'call_AfHEIXajrvBvU0H3x865Pefj': {'name': 'get_weather', 'arguments': '{}'}, 'call_sWp4ie35amsFKvReXAvQ1Mzg': {'name': 'get_weather', 'arguments': '{}'}}
工具调用结束
call_AfHEIXajrvBvU0H3x865Pefj {'name': 'get_weather', 'arguments': '{}'}
call_sWp4ie35amsFKvReXAvQ1Mzg {'name': 'get_weather', 'arguments': '{}'}
